<a href="https://colab.research.google.com/github/azamjon98/test/blob/main/intro_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# prompt:

!git config --global user.email "your.email@example.com"
!git config --global user.name "Your Name"


In [16]:
import pandas as pd
import numpy as np
import sklearn # scikit-learn kutubxonasi
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
# Onlayn dataset joylashgan manzilini ko'rsatamiaz
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


unbalanced splitdan foydalanamiz

In [18]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

balanced split:

In [19]:
df['income_cat'] = pd.cut(df['median_income'], bins=[0., 1.5, 3.0, 4.5, 6.0, np.inf], labels=[1,2,3,4,5])

from sklearn.model_selection import StratifiedShuffleSplit
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# stratified_split.split funksiyasi indekslar qaytaradi
for train_index, test_index in stratified_split.split(df,df['income_cat']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

strat_train_set.drop('income_cat', axis=1, inplace=True)
strat_test_set.drop('income_cat', axis=1, inplace=True)

In [20]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

data setda median house value label vazifasini bajaradi

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   longitude           20640 non-null  float64 
 1   latitude            20640 non-null  float64 
 2   housing_median_age  20640 non-null  float64 
 3   total_rooms         20640 non-null  float64 
 4   total_bedrooms      20433 non-null  float64 
 5   population          20640 non-null  float64 
 6   households          20640 non-null  float64 
 7   median_income       20640 non-null  float64 
 8   median_house_value  20640 non-null  float64 
 9   ocean_proximity     20640 non-null  object  
 10  income_cat          20640 non-null  category
dtypes: category(1), float64(9), object(1)
memory usage: 1.6+ MB


total_bedrooms dagi nan larni o'rnini to'ldiramiz

1) total bedroomsdagi nan larni drop qilamiz

In [22]:
# 1-variant. Qatorlarni tashlab yuborish
housing.dropna(subset=['total_bedrooms']).head() # inplace=True qilishni unutmang

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,INLAND
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN


2) agar ustunda nanlar concentrationi juda ko'p bo'lsa ushbu ustunni tashlab yuboramiz

In [23]:
# 2-variant. Ustunni tashlab yuborish
housing.drop("total_bedrooms", axis=1).head() # inplace=True qilishni unutmang

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,2237.0,706.0,2.1736,INLAND
15502,-117.23,33.09,7.0,5320.0,2015.0,768.0,6.3373,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,667.0,300.0,2.8750,INLAND
14053,-117.13,32.75,24.0,1877.0,898.0,483.0,2.2264,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,1837.0,580.0,4.4964,<1H OCEAN


qandaydir o'rta qiymat bilan to'ldirish: bu yerda o'rta qiymat measure of central tendency dan biri bo'lishi kerak:

In [24]:
# 3-variant. Median qiymat bilan to'ldirish
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median) # inplace=True qilishni unutmang

12655     797.0
15502     855.0
2908      310.0
14053     519.0
20496     646.0
          ...  
15174    1231.0
12661    1422.0
19263     166.0
19140     580.0
19773     222.0
Name: total_bedrooms, Length: 16512, dtype: float64

In [25]:
median

433.0

SimpleImputer classi jarayonlarni avtomatlashtirishga yordam beradi.

In [27]:
# SimpleImputer klassini chaqiramiz
from sklearn.impute import SimpleImputer
# SimpleImputer klassidan yangi obyekt yaratamiz va NaN qiymatlarni median qiymat bilan to'ldirishni ko'rsatamiz
imputer = SimpleImputer(strategy="median")

bu yerda imputer nomli obyekt yaratdik va nan larni to'ldirish usuli sifatida median ni tanladik.

In [28]:
imputer.fit(housing)

ValueError: ignored

fit funksiyasi barcha ustunlar bo'yicha median valueni hisobladi. Lekin ocean_proximity ustuni faqat obyektlardan iborat shunga uning mediani yo'q. shu sababli oxirgi kod bizga value error qaytardi. Uni oldini olish uchun biz data setimizdan faqat numerical qismini ajratib olamiz

In [30]:
housing_num = housing.drop("ocean_proximity", axis=1)
a=imputer.fit(housing_num)

bu yerda fit() hisoblovchi funksiya


In [33]:
a.statistics_  # buyerda statistics metodi a o'zgaruvchiga imuter tomonidan saqlangan arrayni qaytaradi

array([-118.51   ,   34.26   ,   29.     , 2119.     ,  433.     ,
       1164.     ,  408.     ,    3.54155])

In [35]:
# to'ldirish uchun biz transform funksiyasidan foydalanamiz, o'zgaruvchi sifatida data set ni uzatamiz
X = a.transform(housing_num)
#bu yerda X n o'lchovli array

In [37]:
type(X)

numpy.ndarray

In [38]:
# X arrayni data framega o'tkazamiz va uni transformerlangan data deb nomlaymiz
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [39]:
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964


In [40]:
housing_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 12655 to 19773
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
dtypes: float64(8)
memory usage: 1.1 MB


agar bir necha ustunlarda nan qiymatlar bo'lsa u holda oxirgi usul effective, chunki u avtomatik to'ldiradi, agar bitta ustun ozida bo'lsa  fillna() va median() median qiymatlaridan foydalanish afzalroq.

In [41]:
#ML algoritmlari odatda sonlar bilan ishlaydi. shunga matnlarni ham songa o'zgaartirish talab qilinadi.

In [42]:
housing_cat = housing[['ocean_proximity']]

In [43]:
housing_cat["ocean_proximity"].head(10)

12655        INLAND
15502    NEAR OCEAN
2908         INLAND
14053    NEAR OCEAN
20496     <1H OCEAN
1481       NEAR BAY
18125     <1H OCEAN
5830      <1H OCEAN
17989     <1H OCEAN
4861      <1H OCEAN
Name: ocean_proximity, dtype: object

In [44]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]
# OrdinalEncoder() transformatsiyada ordinal sonlardan foydalanadi, mantiqiy jihatdan shifrlamaydi

array([[1.],
       [4.],
       [1.],
       [4.],
       [0.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.]])

yuqorida INLAND=1; NEAR OCEAN=4; <1H OCEAN=0; va island=5; va NEAR BAY=3; bu yerda xech qanday matiqiy bog'liqlik yo'q

bu narsa kelajakda model ni ishlatishta turli xil noaniqliklar va xatoliklarga olib kelishi mumkin.

In [45]:
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [46]:
# Yuqoridagi muammoni oldini olishda boshqa metoddan foydalanamiz
#yuqorida 5 ta kategoriya mavjud. Ushbu usulda xar biri ning qiymatini agar mavjud bo'lsa 1 ga aks xolda 0 ga o'zgartiramiz
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.toarray()

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

usulning kamchiligi har bir kategoriya data setga yangi ustun qo'shadi, va bu yerda data set hajmini kattalashib ketishiga sabab bo'ladi. Ishlashga noqulalik keltirib chiqaradi.


In [47]:
housing_cat.head(10)

,ocean_proximity
12655,INLAND
15502,NEAR OCEAN
2908,INLAND
14053,NEAR OCEAN
20496,<1H OCEAN
1481,NEAR BAY
18125,<1H OCEAN
5830,<1H OCEAN
17989,<1H OCEAN
4861,<1H OCEAN


In [48]:
from sklearn.base import BaseEstimator, TransformerMixin

# bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [54]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=True)
housing_extra_attribs = attr_adder.transform(housing.values)
housing_extra_attribs[0,:]

array([-121.46, 38.52, 29.0, 3873.0, 797.0, 2237.0, 706.0, 2.1736,
       'INLAND', 5.485835694050992, 3.168555240793201,
       0.20578363026077975], dtype=object)

In [53]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
housing_extra_attribs[0,:]

array([-121.46, 38.52, 29.0, 3873.0, 797.0, 2237.0, 706.0, 2.1736,
       'INLAND', 5.485835694050992, 3.168555240793201], dtype=object)

In [55]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

min_max_scaler.fit_transform(housing_num)

array([[0.28784861, 0.63549416, 0.54901961, ..., 0.06261386, 0.13144137,
        0.11542599],
       [0.70916335, 0.05844846, 0.11764706, ..., 0.05639172, 0.14301718,
        0.40257376],
       [0.52888446, 0.30074389, 0.84313725, ..., 0.01861039, 0.05563854,
        0.16379774],
       ...,
       [0.1623506 , 0.62699256, 0.92156863, ..., 0.0127526 , 0.0317401 ,
        0.18481124],
       [0.16434263, 0.61317747, 0.25490196, ..., 0.03377337, 0.09316654,
        0.25492752],
       [0.22011952, 0.78958555, 0.50980392, ..., 0.01743322, 0.03640777,
        0.18151474]])

yuqorida qiymatlarimiz 0 dan 1 gacha o'zgaradi
housing_median_age - 0-50 oralig'ida
total_rooms esa 6-39000 oralig'ida
median_income - 0.5-15 oralig'ida o'zgarar edi endi ular bitta scalede joylashgan
#### 3.4.1 Min-max scaling

Min-max scaling berilgan qiymatlarni 0-1 oralig'iga olib keladi.

Buning uchun biror ustundagi barcha qiymatlarning min ($x_{min}$) va max ($x_{max}$) qiymatlari topiladi. Keyin esa ustundagi har bir qiymat ($x$) quyidagi formula asosida hisoblab chiqiladi:

![](https://arshpreetsingh.files.wordpress.com/2017/03/normal.png)

In [56]:
from sklearn.preprocessing import StandardScaler
standart_scaler = StandardScaler()
standart_scaler.fit_transform(housing_num)

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.73260236,
         0.55628602, -0.8936472 ],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.53361152,
         0.72131799,  1.292168  ],
       [ 0.26758118, -0.1259716 ,  1.22045984, ..., -0.67467519,
        -0.52440722, -0.52543365],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ..., -0.86201341,
        -0.86511838, -0.36547546],
       [-1.56080303,  1.2492109 , -1.1653327 , ..., -0.18974707,
         0.01061579,  0.16826095],
       [-1.28105026,  2.02567448, -0.13148926, ..., -0.71232211,
        -0.79857323, -0.390569  ]])

#### 3.4.2 Standard Scaler - standartlash
StandardScaler biroz boshqa usulda ishlaydi. Sonlarni standartlas hcuhn bizga berilgan ustun bo'yicha o'rta qiymat ($\mu$) va standart cheklanish ($\sigma$) talab qilinadi. Har bir qatordag qiymat esa quydagicha hisoblanadi

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxeMnm1BSa8qyNkCB6wqiYa790XtNxCdM48k_be-9zESUvivv_nuY6SXk8fpJK_psGiA&usqp=CAU)


In [57]:
# biz yuqorida amalga oshirgan encodingni pandas orqali ham qilsak bo'ladi:
housing_onehot = pd.get_dummies(housing['ocean_proximity'])
housing_onehot

,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
12655,0,1,0,0,0
15502,0,0,0,0,1
2908,0,1,0,0,0
14053,0,0,0,0,1
20496,1,0,0,0,0
...,...,...,...,...,...
15174,1,0,0,0,0
12661,0,1,0,0,0
19263,1,0,0,0,0
19140,1,0,0,0,0
